In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate, ParameterGrid

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from imblearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix

# sklearn metrics
from sklearn.metrics import average_precision_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

# Objetivo

A partir dos dados fornecidos, queremos que responda, preferencialmente com gráficos, às seguintes questões:

1. Qual é a temperatura mais baixa de cada um dos Bouys?
  Em que mês geralmente ocorre?
  
2. Onde (lat/long) temos o maior nível de água?
  Em que mês geralmente ocorre?
  
3. Como os comprimentos de onda se correlacionam com a temperatura do mar?
  É possível prever com precisão o Comprimento de Onda, com base no Mar Temperatura e a localização Bouy?

## Entendendo os dados

Uma análise sobre Open Ocean Data do Digital Ocean Institute da Irlanda. 


**Intervalo de tempo:** \
Análise feita num intervalo de 1 ano, do dia 21/09/2021 a 21/09/2022.

## Coleta de dados

**Tides** \
Medidas sobre Marés coletadas por várias bóias no mar da Irlanda

Time - UTC \
Station ID \
Latitude - degrees_north \
Longitude - degrees_east \
Water Level (LAT) - meters \
Water Level (OD Malin) - meters \
Quality Control Flag \

In [0]:
# File location and type
file_location_td = "/FileStore/tables/IrishNationalTideGaugeNetwork_615f_3972_7c4e.csv"

df = pd.read_csv(file_location_td)
df

--------------------------------------------------------------------------- 
 FileNotFoundError Traceback (most recent call last)
 <command-2356231727838477> in <module> 
 2 file_location_td = "/FileStore/tables/IrishNationalTideGaugeNetwork_615f_3972_7c4e.csv" 
 3 
 ----> 4 df = pd . read_csv ( file_location_td ) 
 5 df

 /databricks/python/lib/python3.8/site-packages/pandas/io/parsers.py in read_csv (filepath_or_buffer, sep, delimiter, header, names, index_col, usecols, squeeze, prefix, mangle_dupe_cols, dtype, engine, converters, true_values, false_values, skipinitialspace, skiprows, skipfooter, nrows, na_values, keep_default_na, na_filter, verbose, skip_blank_lines, parse_dates, infer_datetime_format, keep_date_col, date_parser, dayfirst, cache_dates, iterator, chunksize, compression, thousands, decimal, lineterminator, quotechar, quoting, doublequote, escapechar, comment, encoding, dialect, error_bad_lines, warn_bad_lines, delim_whitespace, low_memory, memory_map, float_precision, storage_options) 
 608 kwds . update ( kwds_defaults ) 
 609 
 --> 610 return _read ( filepath_or_buffer , kwds ) 
 611 
 612 

 /databricks/python/lib/python3.8/site-packages/pandas/io/parsers.py in _read (filepath_or_buffer, kwds) 
 460 
 461 # Create the parser. 
 --> 462 parser = TextFileReader ( filepath_or_buffer , ** kwds ) 
 463 
 464 if chunksize or iterator : 

 /databricks/python/lib/python3.8/site-packages/pandas/io/parsers.py in __init__ (self, f, engine, **kwds) 
 817 self . options [ "has_index_names" ] = kwds [ "has_index_names" ] 
 818 
 --> 819 self . _engine = self . _make_engine ( self . engine ) 
 820 
 821 def close ( self ) : 

 /databricks/python/lib/python3.8/site-packages/pandas/io/parsers.py in _make_engine (self, engine) 
 1048 )
 1049 # error: Too many arguments for "ParserBase" 
 -> 1050 return mapping [ engine ] ( self . f , ** self . options ) # type: ignore[call-arg] 
 1051 
 1052 def _failover_to_python ( self ) : 

 /databricks/python/lib/python3.8/site-packages/pandas/io/parsers.py in __init__ (self, src, **kwds) 
 1865 
 1866 # open handles 
 -> 1867 self . _open_handles ( src , kwds ) 
 1868 assert self . handles is not None 
 1869 for key in ( "storage_options" , "encoding" , "memory_map" , "compression" ) : 

 /databricks/python/lib/python3.8/site-packages/pandas/io/parsers.py in _open_handles (self, src, kwds) 
 1360 Let the readers open IOHanldes after they are done with their potential raises . 
 1361 """
 -> 1362 self.handles = get_handle(
 1363 src , 
 1364 "r" , 

 /databricks/python/lib/python3.8/site-packages/pandas/io/common.py in get_handle (path_or_buf, mode, encoding, compression, memory_map, is_text, errors, storage_options) 
 640 errors = "replace" 
 641 # Encoding 
 --> 642 handle = open(
 643 handle , 
 644 ioargs . mode , 

 FileNotFoundError : [Errno 2] No such file or directory: '/FileStore/tables/IrishNationalTideGaugeNetwork_615f_3972_7c4e.csv'

In [0]:
# File location and type
file_location_td = "/FileStore/tables/IrishNationalTideGaugeNetwork_615f_3972_7c4e.csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df_tides = spark.read.format('csv') \
                .option("inferSchema", 'true') \
                .option("header", 'true') \
                .option("sep", ',') \
                .load(file_location_td)

display(df_tides)

time,station_id,latitude,longitude,Water_Level_LAT,Water_Level_OD_Malin,QC_Flag
UTC,null,degrees_north,degrees_east,metres,metres,null
2021-09-14T20:55:00Z,Aranmore Island - Leabgarrow,54.9905,-8.4955,2.448,0.245,1
2021-09-14T21:00:00Z,Aranmore Island - Leabgarrow,54.9905,-8.4955,2.4,0.197,1
2021-09-14T21:05:00Z,Aranmore Island - Leabgarrow,54.9905,-8.4955,2.51,0.307,1
2021-09-14T21:10:00Z,Aranmore Island - Leabgarrow,54.9905,-8.4955,2.484,0.281,1
2021-09-14T21:15:00Z,Aranmore Island - Leabgarrow,54.9905,-8.4955,2.37,0.167,1
2021-09-14T21:20:00Z,Aranmore Island - Leabgarrow,54.9905,-8.4955,2.428,0.225,1
2021-09-14T21:25:00Z,Aranmore Island - Leabgarrow,54.9905,-8.4955,2.631,0.428,1
2021-09-14T21:30:00Z,Aranmore Island - Leabgarrow,54.9905,-8.4955,2.551,0.348,1
2021-09-14T21:35:00Z,Aranmore Island - Leabgarrow,54.9905,-8.4955,2.6,0.397,1


**Waves** \
Medidas sobre Ondas coletadas por várias bóias no mar da Irlanda


Time - UTC\
Station_id \
Latitude - degrees_north \
Longitude - degrees_east \
Peak Period - S\
Peak Direction - degrees_true\
Upcross Period - S\
Significant Wave Height - cm\
Maximum Wave Height - cm\
Sea Temperature - degree_C\
Current Speed - m/s\
Current Direction - degrees_true

In [0]:
# File location and type
file_location_wv = "/FileStore/tables/IWaveBNetwork30Min_834a_dfe2_58ae.csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df_waves = spark.read.format('csv') \
                .option("inferSchema", 'true') \
                .option("header", 'true') \
                .option("sep", ',') \
                .load(file_location_wv) \
                .select('time', 'station_id', 'latitude', 'longitude', 'PeakPeriod', 'PeakDirection', 'UpcrossPeriod', 'SignificantWaveHeight', 'Hmax', 'SeaTemperature', 'MeanCurSpeed', 'MeanCurDirTo')

display(df_waves)

time,station_id,latitude,longitude,PeakPeriod,PeakDirection,UpcrossPeriod,SignificantWaveHeight,Hmax,SeaTemperature,MeanCurSpeed,MeanCurDirTo
UTC,null,degrees_north,degrees_east,s,degrees_true,s,cm,cm,degree_C,m/s,degrees_true
2021-09-14T21:22:00Z,AMETS Berth A Wave Buoy,54.2753,-10.29737,14.29,299.5,11.111,435.0,NaN,16.25,NaN,NaN
2021-09-14T21:52:00Z,AMETS Berth A Wave Buoy,54.2753,-10.29737,14.29,286.9,11.111,421.0,NaN,16.2,NaN,NaN
2021-09-14T22:22:00Z,AMETS Berth A Wave Buoy,54.2753,-10.29737,15.38,291.1,10.526,384.0,NaN,16.2,NaN,NaN
2021-09-14T22:52:00Z,AMETS Berth A Wave Buoy,54.2753,-10.29737,15.38,299.5,11.429,426.0,NaN,16.2,NaN,NaN
2021-09-14T23:22:00Z,AMETS Berth A Wave Buoy,54.2753,-10.29737,13.33,299.5,10.811,429.0,NaN,16.2,NaN,NaN
2021-09-14T23:52:00Z,AMETS Berth A Wave Buoy,54.2753,-10.29737,13.33,286.9,10.526,406.0,NaN,16.25,NaN,NaN
2021-09-15T00:22:00Z,AMETS Berth A Wave Buoy,54.2753,-10.29737,15.38,300.9,10.526,417.0,NaN,16.25,NaN,NaN
2021-09-15T00:52:00Z,AMETS Berth A Wave Buoy,54.2753,-10.29737,16.67,293.9,10.526,429.0,NaN,16.25,NaN,NaN
2021-09-15T01:22:00Z,AMETS Berth A Wave Buoy,54.2753,-10.29737,14.29,288.3,10.0,416.0,NaN,16.2,NaN,NaN


## Limpeza dos dados

In [0]:
def just_check_nan(df):
    # Check NaN Pandas and Numpy
    missing_val_count_by_column = (df.isna().sum())

    columns_with_nan = missing_val_count_by_column[missing_val_count_by_column > 0]
    print('Dados com NaN:')
    print(columns_with_nan)
    print('')
    print('columns_with_nan.shape', columns_with_nan.shape)
    print('')

In [0]:
just_check_nan(df_tides)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2356231727838474> in <module> 
 ----> 1 just_check_nan ( df_tides ) 

 <command-2356231727838473> in just_check_nan (df) 
 1 def just_check_nan ( df ) : 
 2 # Check NaN Pandas and Numpy 
 ----> 3 missing_val_count_by_column = ( df . isna ( ) . sum ( ) ) 
 4 
 5 columns_with_nan = missing_val_count_by_column [ missing_val_count_by_column > 0 ] 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getattr__ (self, name) 
 1798 """
 1799 if name not in self . columns : 
 -> 1800 raise AttributeError(
 1801 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1802 jc = self . _jdf . apply ( name ) 

 AttributeError : 'DataFrame' object has no attribute 'isna'

In [0]:
just_check_nan(df_waves)